<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso1/ciclo3/4_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1e7ctPi8O3bTQoLZaO9ZZjwGr2r8Z93RS" width="100%">

# Gensim y Embeddings
---

En este notebook presentaremos algunas generalidades de la librería `gensim` y la estrategia de extracción de características conocida como _embeddings_. Comenzamos instalando e importando las librerías necesarias:

In [ ]:
!pip install unidecode gensim==4.2.0 scipy==1.10.1

> **NOTA**: Después de la instalación, es posible que Google Colaboratory le pida reiniciar el entorno de ejecución (*RUNTIME*). Puede hacerlo haciendo clic en el botón `RESTART RUNTIME` antes de continuar.
 <img src="https://drive.google.com/uc?export=view&id=1x9WLH8bLR5i6yV-NoOheOmlVX2C9l07F" width="80%">

In [ ]:
import re
import spacy
import matplotlib.pyplot as plt
import pandas as pd
from unidecode import unidecode
from IPython.display import display

In [ ]:
spacy.cli.download("es_core_news_lg")

## **1. Generalidades de Gensim**
---

`gensim` es una librería de código abierto para _Python_ que ofrece múltiples estrategias de _embedding_, modelos de tópicos y corpus típicos para pruebas con modelos de NLP.

<img src="https://drive.google.com/uc?export=view&id=1puqgEoMoHEJTyLKb4-KP6Hc9po3Whekv" width="80%">

Comenzamos instalando `gensim`:

In [ ]:
!pip install gensim

`gensim` dispone de los siguientes módulos:

- `utils`: módulo con funcionalidades generales de `gensim` como persistencia y carga de modelos.
- `matutils`: módulo con funcionalidades matemáticas e interacción con librerías científicas como `numpy` o `scipy`.
- `downloader`: permite la descarga de modelos preentrenados.
- `corpora`: se usa para la carga y manipulación de distintos corpus.
- `models`: contiene distintos modelos y tipos de embeddings.
- `similarities`: contiene distintas medidas de similitud textual y semántica.
- `topic_coherence`: módulo para el análisis de coherencia en modelos no supervisados.
- `scripts`: contiene algunos algoritmos para usar `gensim` como un `cli`.
- `parsing`: provee métodos para extracción de información a partir de textos.

Normalmente, muchos de estos módulos no son requeridos para una aplicación típica de NLP, solamente llegan a ser necesarios cuando necesitamos hacer algo muy específico y personalizado. Los módulos con los que generalmente estaremos trabajando son `downloader`, `models`, `corpora` y `similarities`. Veamos el uso de `gensim` en aplicaciones relacionadas a _embeddings_.

## **2. Conjunto de Datos**
---

En este caso, trabajaremos como corpus el antiguo testamento de la Biblia, ya que es un conjunto de datos en español extenso con palabras muy típicas del lenguaje. Primero lo descargamos:

In [ ]:
!wget 'https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/biblia.txt' -O 'biblia.txt'

Ahora, cargamos el dataset, especificamos como encoding `latin_1` para tomar caracteres especiales de lenguajes derivados del latín:

In [ ]:
with open("biblia.txt", encoding="latin_1") as f:
    text = f.readlines()
display(text[:5])

Definimos una función para preprocesar el texto:

In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")
def preprocess(text, min_len=1, max_len=23):
    # Normalizamos el texto
    norm_text = unidecode(text).lower()

    # Extraemos tokens
    tokens = norm_text.split()

    # Filtramos palabras por longitud
    filtered_tokens = filter(
            lambda token: (
                len(token) >= min_len and
                len(token) <= max_len
                ),
            tokens
        )
    filtered_text = " ".join(filtered_tokens)
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", filtered_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

Limpiamos el conjunto de datos:

In [ ]:
corpus = map(preprocess, text)
corpus = list(filter(lambda doc: len(doc), corpus))
display(corpus[:10])

## **3. Embeddings**
---

Un embedding es una representación vectorial de un elemento en un espacio dimensional reducido. Por lo general, se utilizan embeddings en el campo del aprendizaje automático y en el procesamiento del lenguaje natural. Los embeddings se pueden entrenar para capturar las relaciones entre los elementos de un conjunto de datos, lo que puede ser útil en aplicaciones como la clasificación de texto o la recomendación de productos. En general, un embedding es una forma de representar un elemento de un conjunto de datos en un espacio vectorial de menor dimensión, lo que permite que los algoritmos de aprendizaje automático puedan manejar mejor esos datos.

<img src="https://drive.google.com/uc?export=view&id=1m9pQizY35Tonc86nG5-Tmhcx8gKWj8yX" width="80%">

Existen distintos tipos de _embedding_, por ejemplo, `spacy` calcula un atributo `vector` para cada `Token` en sus documentos, veamos un ejemplo:

In [ ]:
nlp = spacy.load("es_core_news_lg")
doc = nlp(corpus[10])
display(doc)

Veamos la representación vectorial del `Token` número 5:

In [ ]:
display(doc[5].vector)

No obstante, esta representación es general (codifica información del conjunto de noticias en español donde el pipeline fue entrenado), con `gensim` podemos entrenar nuestros propios _embeddings_ usando los modelos más típicos de representación:

### **3.1. Word2Vec**
---

Se trata de un método creado por Google en 2013 que está basado en redes neuronales y busca transformar las palabras en vectores numéricos dentro de un espacio vectorial que capture información _contextual_ y _semántica_.

El enfoque de _Word2Vec_ busca que una palabra guarde información de su contexto, es decir, una palabra viene a representar información de las palabras que la rodean (vecindario). Esto se puede ver desde dos enfoques como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1jIfs_cQKSytMoOsim_sGHX7xstJFPzvT" width="70%">

* **Continuous Bag-of-words (CBOW)**: se trata de un modelo que toma como entrada el contexto de una palabra (vecindario) y trata de predecir la palabra en cuestión.
* **Skip-gram**: se trata de un modelo que toma como entrada una palabra y trata de predecir el contexto (vecindario).

Por ejemplo, de la siguiente frase:

> `"el procesamiento de lenguaje natural en Python"`

Siguiendo el modelo de _Skip-Gram_, tendríamos lo siguiente para codificar la palabra `"lenguaje"`.

* **Entradas**: `"lenguaje"`
* **Salidas**: `["el", "procesamiento", "de", "natural", "en", "Python"]`

El proceso general que describe un modelo de _Word2Vec_ es el siguiente:

1. Construcción del vocabulario a nivel de palabra.
2. Extracción de secuencias de un tamaño dado (contexto).
3. Codificación de cada palabra como un vector one-hot o variables _dummy_.
4. Paso de las codificaciones por la arquitectura de la red neuronal.

Desde `gensim` podemos utilizar este modelo con la clase `Word2Vec`:

In [ ]:
from gensim.models.word2vec import Word2Vec

Dentro de los parámetros más relevantes tenemos:

- `sentences`: corpus tokenizado.
- `vector_size`: tamaño de los vectores que se aprenderán por palabra.
- `window`: tamaño del contexto.
- `min_count`: ignora palabras que tengan una frecuencia de documento menor a este valor.
- `workers`: específica cuántos procesos se usan para el entrenamiento del modelo (se realiza de forma distribuida).
- `sg`: 1 para modelo de tipo _Skip-Gram_, 0 para _CBOW_.
- `alpha`: taza de aprendizaje del modelo (hiper-parámetro).
- `min_alpha`: taza de aprendizaje mínima del modelo.
- `seed`: semilla de números aleatorios para reproducibilidad del modelo.
- `max_vocab_size`: limita el número máximo de palabras a codificar.
- `epochs`: número de iteraciones para el entrenamiento del modelo.

Para entrenar el modelo, necesitamos el corpus tokenizado:

In [ ]:
tokens = list(map(lambda doc: doc.split(), corpus))

Ahora, entrenamos el modelo

In [ ]:
model = Word2Vec(
        sentences = tokens,
        vector_size = 100,
        epochs = 20,
        workers = -1 # específica que se debe usar el número máximo de procesos.
        )

Veamos cómo podemos extraer el vector de una palabra en específico con el atributo `wv`:

In [ ]:
vect = model.wv["tierra"]
display(vect)
display(vect.shape)

Como podemos ver, la palabra `"tierra"` se codifica como un vector de tamaño `100`. También podemos extraer una representación vectorial de todo un documento:

In [ ]:
display(tokens[10])

Veamos las representaciones:

In [ ]:
vects = model.wv[tokens[10]]
display(vects)
display(vects.shape)

Como puede ver, obtuvimos `12` vectores de tamaño `100`, correspondientes a las 12 palabras del documento:

In [ ]:
display(len(tokens[10]))

El atributo `wv` de un modelo de `gensim` contiene vectores anotados `KeyedVectors`:

In [ ]:
display(type(model.wv))

Estos vectores anotados son el resultado final del modelo (un vector por cada palabra) y en muchas oportunidades es lo que necesitamos para extraer características de un texto. No obstante, los `KeyedVectors` no almacenan información de los estados internos del modelo ni de su arquitectura (_Skip-Gram_ o _CBOW_).

Con esto, podemos ver las dos formas de exportar modelos de `gensim`:

- **Modelo completo**: podemos almacenar un modelo completo con el método `save`:

In [ ]:
model.save("model.bin")

Para cargarlo, podemos usar el método `load` de `Word2Vec`:

In [ ]:
model = Word2Vec.load("model.bin")
display(model)

De esta forma, almacenamos el modelo con sus estados y parámetros.

- **Vectores**: podemos almacenar únicamente los vectores del modelo:

In [ ]:
model.wv.save("model.vec")

Para cargarlo, usamos la utilidad de `KeyedVectors` del modelo correspondiente:

In [ ]:
from gensim.models.word2vec import KeyedVectors
model = KeyedVectors.load("model.vec")
display(model)

Guardar un modelo completo da más flexibilidad, no obstante, ocupa mucho más espacio en disco y en memoria a diferencia de guardar únicamente los vectores. Veamos una comparación con `os`:

In [ ]:
import os

Veamos el tamaño del modelo completo:

In [ ]:
display(f"{os.stat('model.bin').st_size / 1024 ** 2:.2f} MB")

Veamos el tamaño de los vectores

In [ ]:
display(f"{os.stat('model.vec').st_size / 1024 ** 2:.2f} MB")

### **3.2. FastText**
---

El modelo _FastText_ es una versión mejorada del modelo _Word2Vec_. Este modelo fue propuesto por Facebook en el año 2015.

La intención de _FastText_ es representar palabras que no están dentro del vocabulario, veamos lo que ocurre con el modelo que teníamos entrenado anteriormente:

In [ ]:
try:
    vect = model.wv["pepe"]
    display(vect)
except Exception as e:
    print(e)

Como podemos ver, es una palabra que no se encuentra en el vocabulario y por ello no tiene un vector asociado.

_FastText_ soluciona este problema al manipular secuencias de N-Grams a nivel de caracter en lugar de secuencias de N-Grams a nivel de palabra. De esta forma, la mayoría de texto bien escrito en un idioma se puede codificar e incluso extrapolar para obtener una representación vectorial. Este modelo a nivel de arquitectura es idéntico al modelo de _Word2Vec_, con la diferencia que el contexto y la codificación se realiza a nivel N-gram como se muestra a continuación:

<img src="https://drive.google.com/uc?export=view&id=1Fjo1lnpnreXLF0tEY5c6SlsmEH2RvsPb" width="100%">

Desde `gensim` podemos usar esta estrategia de representación con la clase `FastText`:

In [ ]:
from gensim.models.fasttext import FastText

El modelo es prácticamente idéntico a _Word2Vec_ con la única diferencia que podemos especificar características de los N-grams:

- `min_n`: longitud mínima de _N-Grams_ a considerar.
- `max_n`: longitud máxima de _N-Grams_ a considerar.

Veamos un ejemplo:

In [ ]:
model = FastText(
        sentences = tokens,
        vector_size = 100,
        epochs = 20,
        workers = -1, # específica que se debe usar el número máximo de procesos.
        min_n = 2,
        max_n = 4
        )

Veamos cómo codificar una palabra:

In [ ]:
vect = model.wv["tierra"]
display(vect)

También podemos codificar palabras fuera del vocabulario:

In [ ]:
vect = model.wv["pepe"]
display(vect)

### **3.3. Doc2Vec**
---

Como pudimos verlo hasta este punto, los modelos _Word2Vec_ y _FastText_ se enfocan en codificar texto a nivel palabra. No obstante, en muchas oportunidades necesitamos codificar todo un documento como un vector.

El modelo _Doc2Vec_ presenta una versión modificada de _Word2Vec_ que aplica para documentos completos. Esto se consigue al codificar un identificador del documento en la generación del modelo, como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1yLRIZBzgJW9ls2Azc25YIdPhVtVtDOCx" width="80%">

En `gensim` podemos usar la clase `Doc2Vec` para definir este modelo:

In [ ]:
from gensim.models.doc2vec import Doc2Vec

También es necesario crear un `TaggedDocument` (documento con su `id`):

In [ ]:
from gensim.models.doc2vec import TaggedDocument

Primero, creamos una lista con todos los documentos etiquetados al asignarles su `id` de documento como `tags`:

In [ ]:
tagged_corpus = [
        TaggedDocument(doc, [i])
        for i, doc in enumerate(tokens)
        ]

Veamos un ejemplo de documento con etiqueta:

In [ ]:
display(tagged_corpus[0])

El entrenamiento de _Doc2Vec_ es idéntico al de _Word2Vec_, la única diferencia es que deben entrar documentos etiquetados con el parámetro `documents` en lugar de `sentences`:

In [ ]:
model = Doc2Vec(
        documents = tagged_corpus,
        vector_size = 100,
        epochs = 20,
        workers = -1 # específica que se debe usar el número máximo de procesos.
        )

Veamos cómo podemos codificar un documento del corpus:

In [ ]:
display(tagged_corpus[0])

Veamos el vector resultante, en este caso usamos el método `infer_vector` y extraemos las palabras con el atributo `words` a partir de un `TaggedDocument`:

In [ ]:
vect = model.infer_vector(tagged_corpus[0].words)
display(vect)
display(vect.shape)

Como se puede ver, obtuvimos una codificación única para todo un documento y no por palabra, lo cual muestra la utilidad de _Doc2Vec_.

## **4. Modelos Pre-Entrenados**
---

Una de las principales desventajas de este tipo de modelos basados en _embeddings_ es que requieren mucho tiempo de entrenamiento y un corpus muy grande para llegar a resultados óptimos. Es por esto que muchas veces se suelen utilizar modelos pre-entrenados y posteriormente reajustarlos para nuestro corpus.

Desde `gensim` disponemos de algunos modelos pre-entrenados sobre corpus masivos, podemos listar los modelos disponibles con información extraída del `api`:

In [ ]:
import gensim.downloader as api

Veamos un listado de los modelos disponibles:

In [ ]:
models = api.info()["models"]
display(models)

Como se puede ver, tenemos distintos metadatos relacionados a los modelos pre-entrenados como:

- Estrategia de preprocesamiento:

In [ ]:
model_meta = models["glove-twitter-25"]
display(model_meta["preprocessing"])

- Tamaño del corpus de entrenamiento:

In [ ]:
display(model_meta["num_records"])

- Corpus de entrenamiento:

In [ ]:
display(model_meta["base_dataset"])

- Tamaño del _embedding_:

In [ ]:
display(model_meta["parameters"])

- Descripción del modelo:

In [ ]:
display(model_meta["description"])

Veamos cómo podemos cargar un modelo pre-entrenado. Vamos a seleccionar uno de la siguiente lista:

In [ ]:
display(models.keys())

Descargamos y cargamos el modelo:

In [ ]:
model = api.load("glove-twitter-25")
display(type(model))

Esto nos permite cargar un vocabulario y los vectores asociados a cada palabra como `KeyedVectors`. Por ejemplo, podemos indexar una palabra en específico del vocabulario:

In [ ]:
vect = model["father"]
display(vect)
display(vect.shape)

Normalmente, los modelos pre-entrenados de `gensim` proveen únicamente los vectores anotados. No obstante, podemos encontrar otros modelos publicados por terceros. Por ejemplo, podemos usar un [modelo _FastText_ pre-entrenado de Facebook sobre el conjunto de datos de _WikiNews_ en Español](https://fasttext.cc/docs/en/crawl-vectors.html). Primero lo descargamos:

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1eCrRIJFR-QdX1wRL_bhg2xifMJPk0VKu

Ahora lo cargamos con `gensim`:

In [ ]:
from gensim.models.fasttext import load_facebook_model
model = load_facebook_model("cc.es.300.bin")

Como podemos ver, cargamos un modelo completo que ya fue entrenado sobre un corpus grande:

In [ ]:
display(type(model))

Podemos extraer vectores de este modelo pre-entrenado al igual que en los casos anteriores:

In [ ]:
vect = model.wv["hola"]
display(vect)
display(vect.shape)

## **5. Similitudes**
---

Una de las aplicaciones más interesantes que tienen los _embeddings_ es que nos permiten comparar semánticamente dos textos por medio de una medida de similitud.

<img src="https://drive.google.com/uc?export=view&id=19lvjtDV29-RvY50U9GMRM42wyOqQf0Cb" width="80%">

Por ejemplo, podemos codificar las siguientes cuatro palabras y ver sus similitudes:

In [ ]:
words = ["religión", "iglesia", "deporte", "futbol"]

Extraemos su representación numérica:

In [ ]:
vects = model.wv[words]
display(vects.shape)

Podemos usar `sklearn` para calcular la similitud entre estas palabras:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

Calculamos la similitud:

In [ ]:
sim = cosine_similarity(vects)
display(sim.shape)

El resultado es una matriz de `(4, 4)` con el valor de la similitud coseno entre cada combinación de palabra, veamos las similitudes de forma gráfica con un mapa de calor:

In [ ]:
import seaborn as sns
fig, ax = plt.subplots()
sns.heatmap(pd.DataFrame(sim, index=words, columns=words), annot=True)
fig.show()

Como podemos ver hay una similitud alta entre `"religion"` e `"iglesia"` y también entre `"deporte"` y `"futbol"`, con esto podemos ver que el modelo está capturando relaciones semánticas entre las palabras.

Adicional a esto, `gensim` nos permite recuperar las palabras del vocabulario más parecidas a una palabra dada con el método `most_similar` de los `KeyedVectors`, veamos un ejemplo:

In [ ]:
word = "iglesia"
similar_words = model.wv.most_similar(word, topn=20)
display(similar_words)

Como podemos ver, el resultado nos muestra palabras que parecidas a nivel textual y semántico. En este caso `gensim` nos da un listado con las 20 palabras más parecidas a `"iglesia"`.

Con el método `most_similar` también podemos aplicar relaciones semánticas de las palabras, veamos el siguiente ejemplo:

> `reina = rey - hombre + mujer`

In [ ]:
similar_words = model.wv.most_similar(
        positive=["rey", "mujer"], negative=["hombre"]
        )
display(similar_words)

## **6. Visualización**
---

Finalmente, podemos usar herramientas de visualización para mostrar relaciones semánticas entre distintas palabras, para ello, vamos a tomar 5 conceptos y a encontrar sus 10 palabras más parecidas:

In [ ]:
words = ["religión", "deporte", "política", "economía", "farándula"]

Extraemos un listado con las 10 palabras más parecidas por cada categoría:

In [ ]:
all_words = []
for word in words:
    most_similar = model.wv.most_similar(word, topn=10)
    all_words.extend(map(lambda case: case[0], most_similar))
display(all_words)

Extraemos los vectores de estas palabras:

In [ ]:
vects = model.wv[all_words]
display(vects.shape)

En este caso tenemos 50 vectores de dimensión 300 (lo cual no es fácil de visualizar). Un enfoque típico para poder visualizar estos datos es utilizar una estrategia de reducción de dimensionalidad para mostrar los vectores proyectados en un plano cartesiano. En este caso, usaremos el modelo _Principal Components Analysis_ (PCA) de `sklearn`:

In [ ]:
from sklearn.decomposition import PCA

Obtenemos los vectores proyectados como 2 dimensiones:

In [ ]:
X = PCA(n_components=2).fit_transform(vects)
display(X.shape)

Finalmente, con estos puntos podemos ver qué tan cerca se encuentran los conceptos que encontramos con una nube de puntos:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1])

for word, x, y in zip(all_words, X[:, 0], X[:, 1]):
    ax.annotate(
            word, xy = (x + 0.1, y + 0.1),
            xytext = (0, 0), textcoords = "offset points"
            )
ax.set_xlabel("$PC_1$")
ax.set_ylabel("$PC_2$")
fig.show()

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Gensim: topic modeling for humans](https://radimrehurek.com/gensim/).
- [FastText: library for efficient text classification and representation learning](https://fasttext.cc/).

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*